In [36]:
import os
import subprocess
import json
from web3 import Web3
from dotenv import load_dotenv
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from constants import *
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy


In [3]:
load_dotenv()
private_key = os.getenv("PRIVATE_KEY")
mnemonic = os.getenv("MNEMONIC")
ETH_key = os.getenv("ETH_KEY")
BTC_key = os.getenv("BTC_KEY")
BTC = 'btc'
ETH = 'eth'
BTCTEST = 'btc-test'
#account_one = Account.from_key(os.getenv("ETH_KEY"))

In [122]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)
address = Web3.toChecksumAddress('0x598607fda464947507b187639389a1b47df602c0')
address2 = Web3.toChecksumAddress('0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0')
w3.eth.getBalance(address2)

0

In [5]:
def derive_wallets(coin,numderive, mnemonic):
    
    command = f'./derive -g --mnemonic="{mnemonic}" --format=json --coin="{coin}" --numderive="{numderive}"'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys

In [72]:
#Setting dictionary of coins to be used in the wallet
coins = {"eth":derive_wallets("eth",3,mnemonic),
         "btc-test":derive_wallets("btc-test",3,mnemonic), 
         "btc":derive_wallets("btc",3,mnemonic)}
print(json.dumps(coins, indent=1))

{
 "eth": [
  {
   "path": "m/44'/60'/0'/0/0",
   "address": "0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0",
   "xprv": "xprvA3ti9wQseP9HMVFd9WdMFBKMkucuQAKL5c9LHxfbPKi16v2oHwp3jQLTXcz96FSQaMwxySi44LYxpEBmAhyzBrECf4xLFvHRXVr51LLYAXA",
   "xpub": "xpub6Gt4ZSwmUkhaZyL6FYAMcKG6JwTPod3BSq4w6M5CwfEyyiMwqV8JHCewNt85UF5P1VkaWR57ZM8VU3RMRCEXmowuSBsmZkgJ2Kopjd2QhG6",
   "privkey": "0x29d51ba291b1eaf093aadfa6a005e202db1f0204cf1878085d08810905da87d8",
   "pubkey": "0237d85fe57a81d5c8645e83f8ccb303ea9a0e35122fd21355c251f026a03fc5e2",
   "pubkeyhash": "303a6b933ab7c639181ed0f7b0f8c84e23657c55",
   "index": 0
  },
  {
   "path": "m/44'/60'/0'/0/1",
   "address": "0x766D529670654A9784f66F19dd9f56cd265Ef9cf",
   "xprv": "xprvA3ti9wQseP9HNHF6oE814gqRW5zJvRShNX8648YyF2Frwje8xWWDw6njEfZkSwggnEk3fn1sursgqGcUsWN3vUAgXB1JMaV8NjvZvaRGvti",
   "xpub": "xpub6Gt4ZSwmUkhaamKZuFf1RpnA47poKtAYjk3grWxaoMnqpXyHW3pUUu7D5xSdqgVSjjwGzfkMevs2396SVLSxv2GYbdf23GaBek4vTzBtCcW",
   "privkey": "0x934264cb5a46db2b8c5efeeb4af3ba

In [113]:
# get the private key
eth_PrivateKey = coins[ETH][0]['privkey']
btc_PrivateKey = coins[BTC][0]['privkey']
btc_test_PrivateKey = coins[BTCTEST][0]['privkey']

In [114]:
eth_PrivateKey

'0x29d51ba291b1eaf093aadfa6a005e202db1f0204cf1878085d08810905da87d8'

In [115]:
def priv_key_to_account(coin,priv_key):
    if coin==ETH:
        return Account.from_key(priv_key)   
    elif coin==BTCTEST:
        return PrivateKeyTestnet(priv_key) 
   

In [116]:
priv_key_to_account(ETH,eth_PrivateKey)

In [58]:
priv_key_to_account(BTCTEST, btc_test_PrivateKey) 

<PrivateKeyTestnet: moB38C4KHdsfgeeTinjeyduApqKJLjUoA5>

In [59]:
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address),
            "chainId":11039
        }
    
    elif coin == BTCTEST:
        
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [120]:
eth_acc = priv_key_to_account(ETH, derive_wallets(ETH,5,mnemonic)[0]['privkey'])
eth_acc

In [85]:
# create a function to send txn
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [86]:
eth_PrivateKey = coins[ETH][0]['privkey']
btc_PrivateKey = coins[BTC][0]['privkey']
btc_test_PrivateKey = coins[BTCTEST][0]['privkey']

In [108]:
eth_PrivateKey

'0x29d51ba291b1eaf093aadfa6a005e202db1f0204cf1878085d08810905da87d8'

In [88]:
btc_test_acc = priv_key_to_account(BTCTEST,btc_test_PrivateKey)

In [89]:
btc_test_acc

<PrivateKeyTestnet: moB38C4KHdsfgeeTinjeyduApqKJLjUoA5>

In [97]:
# create BTC transaction
#recipientAccount = n1u1MQ11bBZvj6JzjzJKxiT5PgCsfc3ZiQ

create_tx(BTCTEST,btc_test_acc,"n1u1MQ11bBZvj6JzjzJKxiT5PgCsfc3ZiQ", 0.01)
#key.create_tx(outputs,unspents=key.get_unspents())

'{"unspents":[{"amount":1271330,"confirmations":0,"script":"76a91453fcb8086bc5d3b68f0e3e3f67b8580394aa0fee88ac","txid":"7277c2445dfc23be9d528e77138daf3dd931dc228b333841615f2574f418caa0","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["n1u1MQ11bBZvj6JzjzJKxiT5PgCsfc3ZiQ",1000000],["moB38C4KHdsfgeeTinjeyduApqKJLjUoA5",248278]]}'

In [98]:
# Send BTC transaction
send_txn(BTCTEST,btc_test_acc,"n1u1MQ11bBZvj6JzjzJKxiT5PgCsfc3ZiQ", 0.01)

0100000001a0ca18f474255f614138338b22dc31d93daf8d13778e529dbe23fc5d44c27772010000006a4730440220786689a53702b43f7ac6ae7b92475936a386de0f6e3bace445577becb9e4c8b902205a7d674ca50b5249d621a9a4fe006aa89b9fbf3b183390770009f558adf4b52a0121025937469e78af3fdd1a1d064488c819da92cefa2f4fb24c790dc1480be36880ffffffffff0240420f00000000001976a914df8e381f4c3ca949c0b5def8fbdaf6407444812688acd6c90300000000001976a91453fcb8086bc5d3b68f0e3e3f67b8580394aa0fee88ac00000000


In [117]:
create_tx(ETH,eth_acc,"0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0", 1000)

{'from': '0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0',
 'to': '0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0',
 'value': 1000,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 0}

In [ ]:
send_txn(ETH, eth_acc,"0x8d597BC91A4D596663FB393Ec65fD61C1B8637a0", 1000)